# Explore Azure AI Foundry Hub and Projects

This notebook explores your Azure AI Foundry hub and associated projects, including:
1. Dumping out details of the hub
2. Displaying the docstring of the hub class that documents what a hub is
3. Listing all the child workspaces of type project

## Setup and Authentication

In [ ]:
import os
import json
import pandas as pd
from pprint import pprint
from dotenv import load_dotenv
from pathlib import Path
import inspect
from IPython.display import display, Markdown, HTML
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient
from azure.ai.ml.entities._workspace._ai_workspaces.hub import Hub
from azure.ai.ml.entities._workspace._ai_workspaces.project import Project
from azure.ai.projects import AIProjectClient
from azure.mgmt.cognitiveservices import CognitiveServicesManagementClient


In [ ]:
# Load environment variables from .env file
# Look for .env in the current directory and parent directory
current_dir = Path(__file__).parent.absolute() if '__file__' in globals() else Path().absolute()
root_dir = current_dir.parent
load_dotenv(dotenv_path=root_dir / ".env")

try:
    credential = DefaultAzureCredential()
    print("✓ Successfully initialized DefaultAzureCredential")
except Exception as e:
    print(f"× Error initializing credentials: {str(e)}")

In [ ]:
subscription_id = os.getenv("AZURE_SUBSCRIPTION_ID")
resource_group = os.getenv("AZURE_RESOURCE_GROUP")

## Hub Class Docstring

Check the docstring of the Hub class for an explanation of what an AI Foundry Hub is.

In [ ]:
hub_docstring = inspect.getdoc(Hub)
print("\n=== Hub Class Documentation:")
print("\n" + hub_docstring if hub_docstring else "No docstring available for Hub class")

## Project Class Docstring

Now check the docstring of the Project class to understand what an AI Foundry project is.

In [ ]:
project_docstring = inspect.getdoc(Project)
print("\n==== Project Class Documentation:")
print("\n" + project_docstring if hub_docstring else "No docstring available for project class")

## Some helper functions

In [ ]:
def get_ml_client(workspace_name):
    """
    Initialize and return an MLClient for the specified workspace.
    
    Args:
        workspace_name: Name of the workspace
        
    Returns:
        MLClient: The initialized client or None if initialization fails
    """
    try:
        if not subscription_id or "<" in subscription_id:
            print("× Error: AZURE_SUBSCRIPTION_ID not set or contains placeholder. Please update your .env file.")
            return None
        elif not resource_group or "<" in resource_group:
            print("× Error: Resource group not provided or contains placeholder.")
            return None
        elif not workspace_name:
            print("× Error: Workspace name not provided or contains placeholder.")
            return None
            
        client = MLClient(
            credential=credential,
            subscription_id=subscription_id,
            resource_group_name=resource_group,
            workspace_name=workspace_name
        )
        print(f"✓ Successfully connected to workspace: {workspace_name}")
        return client
        
    except Exception as e:
        print(f"× Error connecting to workspace {workspace_name}: {str(e)}")
        return None

In [ ]:
def get_workspace_details(workspace_or_id, ml_client=None, title=None):
    """Get and display details for a workspace."""
    if isinstance(workspace_or_id, str):
        # handle project
        parts = workspace_or_id.split('/') 
        workspace = ml_client.workspaces.get(
            name=parts[8],
            resource_group_name=parts[4]
        )
    else:
        # handle hub
        workspace = workspace_or_id
    
    display(Markdown(f"## {title} Details"))
    data = {"Property": [], "Value": []}
    workspace_dict = workspace._to_dict()

    for k, v in sorted(workspace_dict.items()):
        data["Property"].append(k)
        if isinstance(v, (dict, list)):
            data["Value"].append(json.dumps(v, indent=4))
        else:
            data["Value"].append(str(v))

    df = pd.DataFrame(data)
    styled_df = df.style.set_properties(**{
        'white-space': 'pre-wrap', 
        'overflow-wrap': 'break-word',
        'max-width': '800px'
    })
    
    display(styled_df)
    
    summary = {"name": workspace.name, "type": "Hub" if isinstance(workspace, Hub) else "Project"}
    if summary["type"] == "Hub" and "associated_workspaces" in workspace_dict:
        summary["child_projects"] = len(workspace_dict["associated_workspaces"])
    elif "hub_id" in workspace_dict:
        summary["parent_hub"] = workspace_dict["hub_id"].split("/")[-1]
        summary["discovery_url"] = workspace_dict["discovery_url"]
        summary["id"] = workspace_dict["id"]
    
    return workspace, workspace_dict, summary

## Instantiate a handle to the AI Foundry Hub

In [ ]:
hub_name = os.getenv("AZURE_AI_FOUNDRY_HUB_NAME")
hub_ml_client = get_ml_client(hub_name)

## Explore the Hub
Retrieve detailed information about the hub using our helper function.

In [ ]:
hub_info = hub_ml_client.workspaces.get(name=hub_name)
hub, hub_dict, hub_summary = get_workspace_details(hub_info, title="Hub")

## Hub connections
List the hub connections

In [ ]:
connection_list = hub_ml_client.connections.list()
for i, conn in enumerate(connection_list):
    display(Markdown(f"##  Connection #{i+1}: {conn.name}"))
    
    # Convert to dictionary and pretty print
    conn_dict = conn.to_dict() if hasattr(conn, 'to_dict') else vars(conn)
    pprint(conn_dict, width=100, sort_dicts=False)

### Hub projects

Now, let's list all the child workspaces of type project associated with this hub using our helper function.

In [ ]:
display(Markdown(f"## Listing child projects"))
project_summaries = []

if hasattr(hub, 'associated_workspaces') and hub.associated_workspaces:
    for i, workspace_id in enumerate(hub.associated_workspaces):
        workspace, _, summary = get_workspace_details(
            workspace_id, 
            ml_client=hub_ml_client, 
            title=f"Project #{i+1}"
        )
        if workspace and isinstance(workspace, Project):
            project_summaries.append(summary)
    
    if project_summaries:
        display(Markdown(f"### Projects summary"))
        projects_df = pd.DataFrame(project_summaries)
        display(projects_df)
        display(Markdown(f"### Total Projects: {len(project_summaries)}"))
else:
    display(Markdown(f"### No associated workspaces found."))

## Explore project connections

In [ ]:
display(Markdown(f"## Exploring Project Connections"))
for project in project_summaries:
    discovery_url = project.get("discovery_url", "")
    project_id = project.get("id", "")
    project_name = project.get("name", "")
    
    if not discovery_url or not project_id:
        print(f"× Skipping project {project_name} due to missing discovery_url or id")
        continue
    
    # Extract different parts of connection config to assemble the connection string
    hostname = discovery_url.replace("https://", "").replace("/discovery", "")
    id_parts = project_id.split('/')
    subscription_id = id_parts[2] if len(id_parts) > 2 else ""
    resource_group = id_parts[4] if len(id_parts) > 4 else ""
    
    conn_str = f"{hostname};{subscription_id};{resource_group};{project_name}"
    
    display(Markdown(f"### Project: {project_name}"))
    print(f"Connection String: {conn_str}")
    
    try:
        project_client = AIProjectClient.from_connection_string(
            conn_str=conn_str,
            credential=credential
        )
        
        # Get connections for this project
        connections = project_client.connections.list()
        display(Markdown(f"### Connections for {project_name} (found {len(connections)}):"))
        
        for i, connection in enumerate(connections):
            display(Markdown(f"#### Connection #{i+1}: {connection.name}"))
            conn_dict = connection.to_dict() if hasattr(connection, 'to_dict') else vars(connection)
            pprint(conn_dict, width=100, sort_dicts=False)
            
    except Exception as e:
        print(f"× Error working with project {project_name}: {str(e)}")

## Get Project Service Deployments
Use CognitiveServicesManagementClient to get the project model deployments deployed as cognitive services resources.

In [ ]:
def list_service_deployments(cognitive_client, resource_group_name, account_name):
    """Get deployments for a cognitive services account and return as a list of dictionaries."""
    try:
        deployments = cognitive_client.deployments.list(resource_group_name, account_name)
        deployment_list = []
        
        for deployment in deployments:
            deployment_info = {
                "Account": account_name,
                "Deployment": deployment.name,
                "Model": deployment.properties.model.name,
                "Scale Type": deployment.sku.name,
                "Scale Size": deployment.sku.capacity
            }
            deployment_list.append(deployment_info)
            
        return deployment_list
    except Exception as e:
        print(f"× Error listing deployments for account {account_name}: {str(e)}")
        return []

In [ ]:
display(Markdown(f"###  Cognitive Services Accounts and Deployments"))

try:
    all_deployments = []
    
    cognitive_client = CognitiveServicesManagementClient(credential, subscription_id)
    accounts = cognitive_client.accounts.list()

    for account in accounts:
        resource_group_name = account.id.split('/')[4]
        account_name = account.name
        account_type = account.kind
        account_location = account.location
        account_endpoint = account.properties.endpoint
        
        # Get deployments for this account
        deployments = list_service_deployments(cognitive_client, resource_group_name, account_name)
        
        # If no deployments were found, add the account info anyway
        if not deployments:
            all_deployments.append({
                "Account": account_name,
                "Type": account_type,
                "Location": account_location,
                "Endpoint": account_endpoint,
                "Deployment": "N/A",
                "Model": "N/A",
                "Scale Type": "N/A",
                "Scale Size": "N/A"
            })
        else:
            for deployment in deployments:
                deployment["Type"] = account_type
                deployment["Location"] = account_location
                deployment["Endpoint"] = account_endpoint
                all_deployments.append(deployment)
    
    if all_deployments:
        df = pd.DataFrame(all_deployments)
        styled_df = df.style.set_properties(**{
            'white-space': 'pre-wrap',
            'overflow-wrap': 'break-word',
            'max-width': '800px'
        })
        display(styled_df)
    else:
        print("No cognitive services accounts or deployments found.")
    
except Exception as e:
    print(f"× Error listing cognitive services accounts: {str(e)}")